In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torchvision
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torchvision import datasets ,models, transforms
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy

plt.ion()   # interactive mode

#  https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/62840b1eece760d5e42593187847261f/transfer_learning_tutorial.ipynb#scrollTo=dF0_RiFvCP4s


In [2]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomDNADataset(Dataset):
    def __init__(self, feature_file,label_file, transform=None, target_transform=None):
        self.img_labels = pd.read_csv(label_file)
        self.img_features = pd.read_csv(feature_file)
        # pandas read the data 
        self.transform = transform
        self.target_transform = target_transform
        # no transform is used here 
        #self.dic= {'-':0,'A' :2**2 , 'C':2**4,'T':2**6,'G':2**7,'N':0}
        self.dic= {'-':[0,0,0,0],'A' :[1,0,0,0] , 'C':[0,1,0,0],'T':[0,0,1,0],'G':[0,0,0,1]}
        # I learned the sequence of dana from file:///D:/downloads/dna-and-animal-classification.pdf
        #and from http://ircamera.as.arizona.edu/Astr2016/text/nucleicacid1.htm 
        # which showed that there are 4 dominante letters to determine the sequence

    def __len__(self):
        return len(self.img_labels)
        # this is just the number rows or samples in the input data as used by 
        # Dataset class 

    def __getitem__(self, idx):
 ############# DNA transform ######################     
        DNA = self.img_features.iloc[idx,1]
        # so DNA is getting the img_features panda csv read and we are taking the second
        #colmn , the first is just ids . idx is generated by pytorch randomly depending on
        # if we shuffle the data or not and the number of batch size (part of Dataset class which
        # we inheret from )


        n = np.array([])
        Our_pad = (3*22*22)-len(DNA)
        # here I am using 1296 which is 36*36 , I choose this number based on 
        # the maximum individual row I found which was around 1058 ,this is used to 
        # generate a padding which is some how consistent along all .I also chose 
        # 36 so that when we do conv and maxpool we get a nice number which is 
        # dividable by 2 How to fix RuntimeError "Expected object 
        #of scalar type Float but got scalar type Double for argument"
        # here to get the right dim we muliply by 2 the in dim 
        
        for i in range(len(DNA)):# DNA is our row where we itterate 

          if DNA[i] in self.dic:
            mut = self.dic[DNA[i]]
            mut = [x * 2 for x in mut]

            n = np.append(n,mut)
          else:
            n= np.append(n,[0,0,0,0])
          
          
        for pad in range(Our_pad):# we pad the end of the sequence with zeros 
          n =np.append(n,[0,0,0,0])

        
        l = np.array(n , dtype=np.float32)
        # this was very trick where the base type of any np array is float 64 or 
        # double but the base type for torch is float 32 so if we convert 
        # a np array to torch we need to first change it to float32 or we get an error 
        # which says 
        Data_array = torch.from_numpy(l)
        #transfer a np to torch 
        #DNA_dense = Data_array
        # depending on the type of network we can keep it flat or 
        # change the dim (resh
        DNA_image = Data_array.reshape(3,44,44)

        # reshape the dim to be in the form of an image of pix 1 which is 
        # usally the RGB or gray channel , in our case it is gray ,
        # then 36 by 36 which what our padding is doing 
        #for a 3 chennel image insted of 1 


#################################################
        label = self.img_labels.iloc[idx, 1]
        # we get the labels from a different file 

        return DNA_image, label

In [3]:

# n = np.array([])
# Our_pad = (3*22*22)-len(DNA1)
# # here I am using 1296 which is 36*36 , I choose this number based on 
# # the maximum individual row I found which was around 1058 ,this is used to 
# # generate a padding which is some how consistent along all .I also chose 
# # 36 so that when we do conv and maxpool we get a nice number which is 
# # dividable by 2 How to fix RuntimeError "Expected object 
# #of scalar type Float but got scalar type Double for argument"
# j = 0 #index
# for i in DNA1:# DNA is our row where we itterate 
  
#   if i in dic:
#     mut = dic[i]
#     mut[3] = j
#     n = np.append(n,mut)
#   else:
#     n= np.append(n,[1,1,1,j])
#   j+=1
# for pad in range(Our_pad):# we pad the end of the sequence with zeros 
#   n =np.append(n,[0,0,0,0])

# print(n.shape)
# l = np.array(n , dtype=np.float32)
# # this was very trick where the base type of any np array is float 64 or 
# # double but the base type for torch is float 32 so if we convert 
# # a np array to torch we need to first change it to float32 or we get an error 
# # which says 
# Data_array = torch.from_numpy(l)
# #transfer a np to torch 
# #DNA_dense = Data_array
# # depending on the type of network we can keep it flat or 
# # change the dim (resh
# DNA_image = Data_array.reshape(3,44,44)


In [4]:
im = torch.ones(1,36,36)
im.shape
im = im.repeat(3,1,1)

im.shape

l = [0,0,1,0]
l = [x * 2 for x in l]
print(l)

[0, 0, 2, 0]


In [5]:
train_features_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/train_features.csv'
test_features_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/test_features.csv'
train_labels_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/train_labels.csv'
test_labels_path = '/content/drive/MyDrive/Colab Notebooks/deep learning CSCI570/deep learning course/dna-barcode-classification/train_labels - Copy.csv'


full_dataset= CustomDNADataset(feature_file=train_features_path,label_file=train_labels_path)
# so pass the whole set by specifing the path of file to be inputed to our custum class 

Test_dataset = CustomDNADataset(feature_file=test_features_path,label_file=test_labels_path)
# for the test_dataset I am using fake label data which is not accually there just so that I can use 
# my custom class , it has the same len as the train data ( I choped some in csv file )

In [6]:
len(full_dataset)

12906

In [7]:
train_size = int(0.80 * len(full_dataset))
# this is a creative way to create a validation set by spliting the dataset 
validation_size = len(full_dataset) - train_size
train_dataset, validation_dataset = torch.utils.data.random_split(full_dataset, [train_size, validation_size])


In [8]:
dataloaders = {'train': torch.utils.data.DataLoader(train_dataset, batch_size=15,
                                             shuffle=True, num_workers=2),
               'val' : torch.utils.data.DataLoader(validation_dataset, batch_size=15,
                                             shuffle=True, num_workers=2)}
dataset_sizes = {'train': len(train_dataset) ,
                 'val' : len(validation_dataset)}





In [9]:
# set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
 


# hyper parameters 
input_size = 36*36
in_channel = 1
num_classes = 1214
learning_rate = 0.001
batch_size = 15
# the batch size appered to be a significant factor on the effectivness of the training
# 15 is found to be a good num 
num_epochs = 3
#


In [10]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [11]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['val']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title('predicted: {}'.format(class_names[preds[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [12]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.
# Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names)).
model_ft.fc = nn.Linear(num_ftrs, num_classes)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [13]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


train Loss: 5.2824 Acc: 0.2501
val Loss: 3.1378 Acc: 0.5411

Epoch 1/24
----------
train Loss: 2.3560 Acc: 0.7184
val Loss: 1.4104 Acc: 0.8280

Epoch 2/24
----------
train Loss: 1.0069 Acc: 0.9087
val Loss: 0.6595 Acc: 0.9191

Epoch 3/24
----------
train Loss: 0.4815 Acc: 0.9592
val Loss: 0.4113 Acc: 0.9454

Epoch 4/24
----------
train Loss: 0.2519 Acc: 0.9801
val Loss: 0.3104 Acc: 0.9547

Epoch 5/24
----------
train Loss: 0.1446 Acc: 0.9906
val Loss: 0.2771 Acc: 0.9578

Epoch 6/24
----------
train Loss: 0.0885 Acc: 0.9956
val Loss: 0.2590 Acc: 0.9613

Epoch 7/24
----------
train Loss: 0.0572 Acc: 0.9981
val Loss: 0.2536 Acc: 0.9617

Epoch 8/24
----------
train Loss: 0.0516 Acc: 0.9985
val Loss: 0.2525 Acc: 0.9609

Epoch 9/24
----------
train Loss: 0.0477 Acc: 0.9987
val Loss: 0.2494 Acc: 0.9605

Epoch 10/24
----------
train Loss: 0.0466 Acc: 0.9986
val Loss: 0.2486 Acc: 0.9613

Epoch 11/24
----------
train Loss: 0.0433 Acc: 0.9989
val Loss: 0.2476 Acc: 0.9617

Epoch 12/24
----------
t

KeyboardInterrupt: ignored

In [14]:
test_loader = DataLoader(Test_dataset  , batch_size=1 , shuffle=False)
def write_csv_results (loader,model):
    num_correct = 0
    num_samples = 0
    model.eval()
    your_file = open('resultsWithforeign_75_transferLearningCNN.csv', 'ab')
    # we create a csv file or open it 'ab' for append binery 
    sm = torch.nn.Softmax()
    # we use a softmax to compute the probability of each guess of the network 
    
    

    with torch.no_grad():
      for x,_ in loader:
        x = x.to(device = device )
       
        
        scores = model(x)
        _,prediction = scores.max(1)
        #print(scores.max())
        #print('model prediction = ',prediction , 'y = ',y)
        probabilities = sm(scores) 
        #print(probabilities.max())
        #print(prediction)
        if probabilities.max() < 0.95 :
          prediction = torch.tensor([-1])
          np.savetxt(your_file,prediction.cpu())
          # we save the prediction in the open file but first we have to 
          # send it back to being a cpu 
        else:
          np.savetxt(your_file,prediction.cpu().numpy())  
        
        

      model.train()
      your_file.close()

write_csv_results(test_loader,model_ft)
# using our def 


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [ ]:
#model = NN(input_size,num_classes).to(device)

In [ ]:
torch.save(model_ft.state_dict(), '/content/transfer_learning_firstrun_RGB_imagenet_model.pt')
# save the model to the Path which saves the state_dict(from url https://pytorch.org/tutorials/beginner/saving_loading_models.html)